#CEARUN Visualization

---



In [ ]:
pip install beautifulsoup4 lxml

In [ ]:
# @title Imports
import requests
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.ticker import MultipleLocator, FuncFormatter
from cycler import cycler


# Extract The Data From the Website

Change the HTML link for each test from the website [CEARUN](https://cearun.grc.nasa.gov/)

In [ ]:
# Specify the URL of the HTML page
url = 'https://cearun.grc.nasa.gov/OFILES/_______________9399.html'

In [ ]:
# @title Reading the HTML File
# Fetch the HTML content from the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'lxml')

    # Print the title of the HTML document
    print(f"Title: {soup.title.string}")

    # Extract and print all paragraph tags
    paragraphs = soup.find_all('pre')
    for idx, paragraph in enumerate(paragraphs, start=1):
        print("Extracted Text")
else:
    print(f"Failed to retrieve the HTML content. Status code: {response.status_code}")


Failed to retrieve the HTML content. Status code: 404


In [ ]:
# @title Finding the Different Pressures
# Find the main content element (assuming it's within a <pre> tag)
main_content = soup.find('pre').get_text()

# Split the content into lines
lines = main_content.split('\n')

# Flag to indicate when to start capturing lines
capture = False

# List to hold the captured lines
captured_lines = []

#Create an Array For PSIA Values
pressure_pattern = r"p,psia= ([\d\s,]+)"
pressure_match = re.search(pressure_pattern, main_content)
if pressure_match:
    pressures = [int(p.strip()) for p in pressure_match.group(1).split(',')]
    print("Pressures:", pressures)


NameError: name 'soup' is not defined

In [ ]:
# @title Parsing and Creating A DataFrame
def extract_data(text):
    # Define the pattern to match the required data
    pattern = r"Pin =\s+([\d\.]+)\s+PSIA.*?O/F=\s+([\d\.]+).*?T, K\s+([\d\.]+).*?CSTAR, M/SEC\s+([\d\.]+).*?Isp, M/SEC\s+((?:[\d\.]+\s+){6}[\d\.]+)"

    # Find all matches in the text
    matches = re.findall(pattern, text, re.DOTALL)

    # Extract and structure the data
    data = []
    for match in matches:
        pressure = float(match[0])
        of_ratio = float(match[1])
        chamber_temp = float(match[2])
        cstar = float(match[3])
        isp_values = [float(value) for value in match[4].split()]
        data.append({
            'Pressure (PSIA)': pressure,
            'O/F Ratio': of_ratio,
            'Chamber Temp (K)': chamber_temp,
            'CSTAR (m/s)': cstar,
            'Isp (s)': isp_values
        })

    return data

# Example usage
text = main_content

# Extract data from text
data = extract_data(text)

# Convert to pandas DataFrame
df = pd.DataFrame(data)

indexes_to_drop = []
i = 0
while i < len(df):
    pressure = df.loc[i, 'Pressure (PSIA)']
    if pressure in pressures:
        position = pressures.index(pressure)
        isp_values = df.at[i, 'Isp (s)']
        if isinstance(isp_values, float):  # Check if isp_values is a float
            isp_values = [isp_values]  # Convert to list if it's a float
        #Assigning the right ISP Value Based
        if position < len(pressures):
            if position < 6:
                df.at[i, 'Isp (s)'] = isp_values[position + 1] / 9.80665
                if i + 1 < len(df) and len(pressures) > 6:
                    indexes_to_drop.append(i + 1)
                    i += 2  # Skip the next index
                else:
                  i += 1
            elif(position >= 6 and i < len(df) - 1):
                df.at[i+1, 'Isp (s)'] = df.at[i+1, 'Isp (s)'][(position % 6) + 1] / 9.80665
                indexes_to_drop.append(i)
                i += 2  # Skip the next two indexes
        else:
            i += 1
    else:
        i += 1

df = df.drop(indexes_to_drop)
df.reset_index(drop=True, inplace=True)

print(df.head(12), '\n', '\n', df.tail(12))



---


# Chamber Temperature Graphs

In [ ]:
# @title O/F Ratio vs Chamber Temperature for Different Pressures
#Creates the colors for each plot
colors = cycler('color', ['b', 'g', 'r', 'c', 'm', 'y', 'darkred', 'orangered', 'purple', 'pink', 'darkgreen', 'darkorange', 'lightblue'])
plt.rc('axes', prop_cycle=colors) #Adds the colors to the graph

plt.figure(figsize=(5, 5)) #figsize = (length, width)

# Plotting the data
for pressure in df['Pressure (PSIA)'].unique():
    subset = df[df['Pressure (PSIA)'] == pressure]
    plt.scatter(subset['O/F Ratio'], subset['Chamber Temp (K)'], label=f'Pin = {pressure} PSIA', s=12) # s=num is the size of the points
    plt.plot(subset['O/F Ratio'], subset['Chamber Temp (K)'], lw = 1) # lw = num is the width of the plot line

plt.xticks(df['O/F Ratio'].unique())
plt.gca().yaxis.set_major_locator(MultipleLocator(10)) #the number represents the scale of the graph

# Custom formatter to label every other tick on the y-axis
#adjust the y % num == 0 to change the amount of labels on the y-axis
#y % 50 will count by 50s
def custom_formatter(y, pos):
    if y % 50 == 0:
        return f'{y}'
    else:
        return ''

plt.gca().yaxis.set_major_formatter(FuncFormatter(custom_formatter)) #adds the custom axis labels to the graph

plt.xlabel('O/F Ratio')
plt.ylabel('Chamber Temp (K)')
plt.title('O/F Ratio vs Chamber Temperature for Different Pressures')
plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
plt.grid(True)

In [ ]:
# @title Individual Chamber Temperature for Different Pressures
unique_pressures = df['Pressure (PSIA)'].unique()
n = len(unique_pressures)

fig, axs = plt.subplots(4, 3, figsize=(12, 12), sharey=True) #Will generate a 4x3 (lxw) table for the graphs with a size 12
axs = axs.flatten()  # Flatten the 2D array of axes to make indexing easier

#Plotting the Data
for i, pressure in enumerate(unique_pressures):
    subset = df[df['Pressure (PSIA)'] == pressure]
    axs[i].scatter(subset['O/F Ratio'], subset['Chamber Temp (K)'], label=f'Pin = {pressure} PSIA', s=12)
    axs[i].plot(subset['O/F Ratio'], subset['Chamber Temp (K)'], lw=1)
    axs[i].yaxis.set_major_locator(MultipleLocator(25)) #Change number to change the scale of y-axis
    axs[i].yaxis.set_major_formatter(FuncFormatter(custom_formatter)) #Uses the same formatter of the first graph to remove axis labels
    axs[i].set_xticks(subset['O/F Ratio']) #Sets x-axis to match the number of o/f ratios
    axs[i].set_xlabel('O/F Ratio')
    axs[i].set_ylabel('Chamber Temp (K)')
    axs[i].set_title(f'Pressure = {pressure} PSIA')
    axs[i].grid(True)

for j in range(i+1, len(axs)):
    fig.delaxes(axs[j])

plt.xticks(df['O/F Ratio'].unique())
plt.tight_layout()
plt.show()

In [ ]:
# @title Pressure vs Chamber Temperature for Different O/F Ratios
# Plotting the data
scatter_handles =[]
plt.rc('axes', prop_cycle=colors)

plt.figure(figsize=(5, 5))
for of_ratio in df['O/F Ratio'].unique():
    subset = df[df['O/F Ratio'] == of_ratio]
#Creates the colors for each plot
    scatter = plt.scatter(subset['Pressure (PSIA)'], subset['Chamber Temp (K)'], label=f'O/F = {of_ratio}', s=12) # s=num is the size of the points
    plt.plot(subset['Pressure (PSIA)'], subset['Chamber Temp (K)'], label=f'O/F = {of_ratio}', lw=1)  # lw = num is the width of the plot line
    scatter_handles.append(scatter)

plt.xlabel('Pressure (PSIA)')
plt.ylabel('Chamber Temp (K)')
plt.title('Pressure (PSIA) vs Chamber Temperature for Different O/F')
plt.xticks(df['Pressure (PSIA)'].unique())
plt.gca().yaxis.set_major_locator(MultipleLocator(10)) #Scales y-axis by 10s
plt.gca().yaxis.set_major_formatter(FuncFormatter(custom_formatter)) #Uses the first formatter (by 50's)
plt.legend(handles=scatter_handles, bbox_to_anchor=(1, 1), loc='upper left')
plt.grid(True)
plt.show()

In [ ]:
# @title Individual Chamber Temperature for Different O/F Ratios
unique_of = df['O/F Ratio'].unique()
n = len(unique_of)

fig, axs = plt.subplots(5, 3, figsize=(12, 12), sharey=True) #Arranges the graphs by 5x3
axs = axs.flatten()  # Flatten the 2D array of axes to make indexing easier

for i, of_ratio in enumerate(unique_of[:len(axs)]):  # Limit to the number of subplots
    subset = df[df['O/F Ratio'] == of_ratio]
    axs[i].scatter(subset['Pressure (PSIA)'], subset['Chamber Temp (K)'], label=f'O/F Ratio = {of_ratio}', s=12) #s = size of points
    axs[i].plot(subset['Pressure (PSIA)'], subset['Chamber Temp (K)'], lw=1) #lw = width of plot line
    axs[i].yaxis.set_major_locator(MultipleLocator(50)) #counts by 50s
    axs[i].yaxis.set_major_formatter(FuncFormatter(custom_formatter))
    axs[i].set_xlabel('Pressure (PSIA)')
    axs[i].set_ylabel('Chamber Temp (K)')
    axs[i].set_title(f'Chamber Temperature for O/F {of_ratio}')
    axs[i].grid(True)

# If there are more subplots than data points, hide the extra subplots
for j in range(i+1, len(axs)):
    fig.delaxes(axs[j])

plt.xticks(df['Pressure (PSIA)'].unique())

plt.tight_layout()
plt.show()



---

#ISP Graphs

In [ ]:
# @title O/F Ratio vs ISP For Different Pressures

scatter_handles =[]
plt.rc('axes', prop_cycle=colors) #adds the colors to the graph
plt.figure(figsize=(5, 5))

# Plotting the data
for pressure in df['Pressure (PSIA)'].unique():
    subset = df[df['Pressure (PSIA)'] == pressure]
    scatter = plt.scatter(subset['O/F Ratio'], subset['Isp (s)'], label=f'Pin = {pressure} PSIA', s=12) #s = size of points
    plt.plot(subset['O/F Ratio'], subset['Isp (s)'], label=f'Pin = {pressure} PSIA', lw=1) #lw = size of the line
    scatter_handles.append(scatter)

# Custom formatter to label every other tick on the y-axis
#adjust the y % num == 0 to change the amount of labels on the y-axis
#y % 50 will count by 50s
def custom_formatter2(y, pos):
    if y % 5 == 0:
        return f'{y}'
    else:
        return ''

plt.gca().yaxis.set_major_locator(MultipleLocator(1)) # scale of the axis is by 10s
plt.gca().yaxis.set_major_formatter(FuncFormatter(custom_formatter2))
plt.xticks(df['O/F Ratio'].unique())
plt.xlabel('O/F Ratio')
plt.ylabel('Isp (s)')
plt.title('O/F Ratio vs Isp (s) for Different Pressures')
plt.legend(handles=scatter_handles, bbox_to_anchor=(1, 1), loc='upper left')
plt.grid(True)
plt.show()

In [ ]:
# @title Individual ISP For Different Pressures { display-mode: "form" }
unique_pressures = df['Pressure (PSIA)'].unique()
n = len(unique_pressures)

fig, axs = plt.subplots(4, 3, figsize=(12, 12), sharey=True) # Displays the graphs in a 4x3 array
axs = axs.flatten()  # Flatten the 2D array of axes to make indexing easier

for i, pressure in enumerate(unique_pressures):
    subset = df[df['Pressure (PSIA)'] == pressure]
    axs[i].scatter(subset['O/F Ratio'], subset['Isp (s)'], label=f'Pin = {pressure} PSIA', s=12) # s = size of the points
    axs[i].plot(subset['O/F Ratio'], subset['Isp (s)'], lw=1) #lw = width of the plot line
    axs[i].yaxis.set_major_locator(MultipleLocator(1)) # y-axis scale is by 10
    axs[i].yaxis.set_major_formatter(FuncFormatter(custom_formatter2))
    axs[i].set_xticks(subset['O/F Ratio'])
    axs[i].set_xlabel('O/F Ratio')
    axs[i].set_ylabel('Isp (s)')
    axs[i].set_title(f'Pressure = {pressure} PSIA')
    axs[i].grid(True)

# If there are more subplots than data points, hide the extra subplots
for j in range(i+1, len(axs)):
    fig.delaxes(axs[j])
plt.tight_layout()
plt.show()

In [ ]:
# @title Pressure vs ISP For Different O/F Ratios
plt.rc('axes', prop_cycle=colors) #Adds the colors to the graph
scatter_handles =[]
plt.figure(figsize=(5, 5)) # Size of graph

# Plotting the data
for of_ratio in df['O/F Ratio'].unique():
    subset = df[df['O/F Ratio'] == of_ratio]
    scatter = plt.scatter(subset['Pressure (PSIA)'], subset['Isp (s)'], label=f'O/F = {of_ratio}', s=12) #s = size of the points
    plt.plot(subset['Pressure (PSIA)'], subset['Isp (s)'], label=f'O/F = {of_ratio}', lw=1) # lw = width of the line
    scatter_handles.append(scatter)
plt.xlabel('Pressure (PSIA)')
plt.ylabel('Isp (s)')
plt.title('Pressure (PSIA) vs Isp (s) for Different O/F')
plt.gca().yaxis.set_major_locator(MultipleLocator(1)) #scales yaxis by 10
plt.gca().yaxis.set_major_formatter(FuncFormatter(custom_formatter2))
plt.legend(handles=scatter_handles, bbox_to_anchor=(1, 1), loc='upper left')
plt.xticks(df['Pressure (PSIA)'].unique())
plt.grid(True)
plt.show()

In [ ]:
# @title Individual ISP For Different O/F Ratios
fig, axs = plt.subplots(5, 3, figsize=(12, 12), sharey=True) #Creates an array of 5x3 of size 12 Graphs
axs = axs.flatten()  # Flatten the 2D array of axes to make indexing easier

for i, of_ratio in enumerate(unique_of[:len(axs)]):  # Limit to the number of subplots
    subset = df[df['O/F Ratio'] == of_ratio]
    axs[i].scatter(subset['Pressure (PSIA)'], subset['Isp (s)'], label=f'O/F Ratio = {of_ratio}', s=12) #s = size of the point
    axs[i].plot(subset['Pressure (PSIA)'], subset['Isp (s)'], lw=1) # lw = width of the plot line
    axs[i].yaxis.set_major_locator(MultipleLocator(1)) # scales y-axis by 10
    axs[i].yaxis.set_major_formatter(FuncFormatter(custom_formatter2))
    axs[i].set_xlabel('Pressure (PSIA)')
    axs[i].set_ylabel('Isp (s)')
    axs[i].set_title(f'Isp (s) for O/F {of_ratio}')
    axs[i].grid(True)
    axs[i].set_xticks(subset['Pressure (PSIA)'])  # Set x-axis ticks based on Pressure (PSIA) values


# If there are more subplots than data points, hide the extra subplots
for j in range(i+1, len(axs)):
    fig.delaxes(axs[j])

plt.xticks(df['Pressure (PSIA)'].unique())
plt.tight_layout()
plt.show()



---

#CSTAR Graphs

In [ ]:
# @title O/F Ratio vs CSTAR For Different Pressures
scatter_handles =[]
plt.rc('axes', prop_cycle=colors)
plt.figure(figsize=(5, 5))

# Plotting the data
for pressure in df['Pressure (PSIA)'].unique():
    subset = df[df['Pressure (PSIA)'] == pressure]
    scatter = plt.scatter(subset['O/F Ratio'], subset['CSTAR (m/s)'], label=f'Pin = {pressure} PSIA', s=12) #s = size of the points
    plt.plot(subset['O/F Ratio'], subset['CSTAR (m/s)'], label=f'Pin = {pressure} PSIA', lw = 1) # lw = width of the plot line
    scatter_handles.append(scatter)

plt.xlabel('O/F Ratio')
plt.ylabel('CSTAR (m/s)')
plt.title('O/F Ratio vs CSTAR for Different Pressures')

#Creating a new custom formatter to adjust for the scale of this graph
#Change the y % 10 to adjust the scale of the labels shown
def custom_formatter3(y, pos):
    if y % 10 == 0:
        return f'{y}'
    else:
        return ''

plt.xticks(df['O/F Ratio'].unique())
plt.gca().yaxis.set_major_locator(MultipleLocator(2)) # scaling y-axis by 2
plt.gca().yaxis.set_major_formatter(FuncFormatter(custom_formatter3))
plt.legend(handles=scatter_handles, bbox_to_anchor=(1, 1), loc='upper left')
plt.grid(True)
plt.show()

In [ ]:
# @title Individual CSTAR For Different Pressures
fig, axs = plt.subplots(4, 3, figsize=(12, 12), sharey=True) #Creates an array of 4x3 of size 12 graphs
axs = axs.flatten()  # Flatten the 2D array of axes to make indexing easier

#Plotting the Data
for i, pressure in enumerate(unique_pressures):
    subset = df[df['Pressure (PSIA)'] == pressure]
    axs[i].scatter(subset['O/F Ratio'], subset['CSTAR (m/s)'], label=f'Pin = {pressure} PSIA', s=12) #s = size of the points
    axs[i].plot(subset['O/F Ratio'], subset['CSTAR (m/s)'], lw=1) #lw = width of the plot line
    axs[i].yaxis.set_major_locator(MultipleLocator(2)) # scales y-axis by 2
    axs[i].yaxis.set_major_formatter(FuncFormatter(custom_formatter3))
    axs[i].set_xticks(subset['O/F Ratio'])
    axs[i].set_xlabel('O/F Ratio')
    axs[i].set_ylabel('CSTAR (m/s))')
    axs[i].set_title(f'Pressure = {pressure} PSIA')
    axs[i].grid(True)

# If there are more subplots than data points, hide the extra subplots
for j in range(i+1, len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()

In [ ]:
# @title Pressure vs CSTAR For Different O/F Ratios
scatter_handles =[]
plt.rc('axes', prop_cycle=colors)
plt.figure(figsize=(5, 5))

# Plotting the data
for of_ratio in df['O/F Ratio'].unique():
    subset = df[df['O/F Ratio'] == of_ratio]
    scatter = plt.scatter(subset['Pressure (PSIA)'], subset['CSTAR (m/s)'], label=f'O/F = {of_ratio}', s=12) #s = size of the point
    plt.plot(subset['Pressure (PSIA)'], subset['CSTAR (m/s)'], label=f'O/F = {of_ratio}', lw = 1) #lw = width of the plot line
    scatter_handles.append(scatter)


plt.xlabel('Pressure (PSIA)')
plt.ylabel('CSTAR (m/s)')
plt.title('Pressure (PSIA) vs CSTAR for Different O/F')
plt.gca().xaxis.set_major_locator(MultipleLocator(20)) #scales x-axis by 20s
plt.gca().yaxis.set_major_locator(MultipleLocator(2)) #scales y-axis by 2s
plt.gca().yaxis.set_major_formatter(FuncFormatter(custom_formatter3))
plt.legend(handles=scatter_handles, bbox_to_anchor=(1, 1), loc='upper left')
plt.grid(True)
plt.xticks(df['Pressure (PSIA)'].unique())
plt.show()

In [ ]:
# @title Individual CSTAR For Different O/F Ratios
fig, axs = plt.subplots(5, 3, figsize=(12, 12), sharey=True)
axs = axs.flatten()  # Flatten the 2D array of axes to make indexing easier

#Plotting the Data
for i, of_ratio in enumerate(unique_of[:len(axs)]):  # Limit to the number of subplots
    subset = df[df['O/F Ratio'] == of_ratio]
    axs[i].scatter(subset['Pressure (PSIA)'], subset['CSTAR (m/s)'], label=f'O/F Ratio = {of_ratio}', s=12) #s= size of the points
    axs[i].plot(subset['Pressure (PSIA)'], subset['CSTAR (m/s)'], lw=1) #lw = width of the plot line
    axs[i].yaxis.set_major_locator(MultipleLocator(5)) # scales y-axis by 5s
    axs[i].yaxis.set_major_formatter(FuncFormatter(custom_formatter3))
    axs[i].set_xlabel('Pressure (PSIA)')
    axs[i].set_ylabel('CSTAR (m/s)')
    axs[i].set_title(f'CSTAR (m/s) for O/F {of_ratio}')
    axs[i].grid(True)
    axs[i].set_xticks(subset['Pressure (PSIA)'])

# If there are more subplots than data points, hide the extra subplots
for j in range(i+1, len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()
